In [1]:
# Gerekli kütüphaneler
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
# Cihaz kontrolü (GPU Kullanımı)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
input_dim = 28   # Her bir satır (zaman adımı)
hidden_dim = 128 # Daha büyük gizli katman boyutu
layer_dim = 2    # 2 Katmanlı RNN
output_dim = 10  # 10 sınıf (0-9 arası rakamlar)
sequence_length = 28  # 28 zaman adımı
num_epochs = 5
batch_size = 100
learning_rate = 0.001  # Daha stabil öğrenme için düşük LR

In [4]:
# MNIST Veri Kümesini Yükleme ve Normalizasyon
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [5]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [6]:
# RNN Modeli
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        # Katman ve Boyutları
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        
        # RNN Katmanı (tanh fonksiyonuyla daha stabil çalışır)
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='tanh')
        
        # Tam Bağlantılı (Fully Connected) Çıkış Katmanı
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # Başlangıç gizli durumu (GPU için .to(device) ekledik)
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device)
        
        # RNN ileri yayılım (forward propagation)
        out, _ = self.rnn(x, h0)  # out: (batch_size, seq_length, hidden_dim)
        
        # Son zaman adımındaki çıktıyı al ve tam bağlı katmana ver
        out = self.fc(out[:, -1, :])  # Son zaman adımındaki çıktı
        return out


In [7]:
# Model, Loss ve Optimizer Tanımlama
model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# Model Eğitimi
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, sequence_length, input_dim).to(device)  # [batch_size, seq_length, input_size]
        labels = labels.to(device)

        # Gradients sıfırlama
        optimizer.zero_grad()

        # Forward propagation
        outputs = model(images)

        # Loss hesaplama
        loss = criterion(outputs, labels)

        # Backward propagation
        loss.backward()

        # Parametre güncelleme
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

Epoch [1/5], Step [100/600], Loss: 0.9283
Epoch [1/5], Step [200/600], Loss: 0.8463
Epoch [1/5], Step [300/600], Loss: 0.5759
Epoch [1/5], Step [400/600], Loss: 0.5905
Epoch [1/5], Step [500/600], Loss: 0.4137
Epoch [1/5], Step [600/600], Loss: 0.4265
Epoch [2/5], Step [100/600], Loss: 0.2646
Epoch [2/5], Step [200/600], Loss: 0.3827
Epoch [2/5], Step [300/600], Loss: 0.2570
Epoch [2/5], Step [400/600], Loss: 0.1381
Epoch [2/5], Step [500/600], Loss: 0.2380
Epoch [2/5], Step [600/600], Loss: 0.2934
Epoch [3/5], Step [100/600], Loss: 0.3423
Epoch [3/5], Step [200/600], Loss: 0.1699
Epoch [3/5], Step [300/600], Loss: 0.3209
Epoch [3/5], Step [400/600], Loss: 0.0787
Epoch [3/5], Step [500/600], Loss: 0.1850
Epoch [3/5], Step [600/600], Loss: 0.2663
Epoch [4/5], Step [100/600], Loss: 0.1304
Epoch [4/5], Step [200/600], Loss: 0.1957
Epoch [4/5], Step [300/600], Loss: 0.1674
Epoch [4/5], Step [400/600], Loss: 0.1196
Epoch [4/5], Step [500/600], Loss: 0.1241
Epoch [4/5], Step [600/600], Loss:

In [9]:
# Modeli Test Etme
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.view(-1, sequence_length, input_dim).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

print("Eğitim Tamamlandı!")

Test Accuracy: 95.98%
Eğitim Tamamlandı!
